In [ ]:
#LIBRERIAS A USAR
import pandas as pd
import numpy as np
import os


In [ ]:
movies_df_filepath = os.path.abspath("dataset/movies_original.csv")
ratings_df_filepath = os.path.abspath("dataset/ratings_original.csv")
poster_df_filepath = os.path.abspath("dataset/MovieGenre.csv")

#CARGA DE LOS DATASETS
movies_df = pd.read_csv(movies_df_filepath)
ratings_df = pd.read_csv(ratings_df_filepath)
poster_df = pd.read_csv(poster_df_filepath, encoding='latin-1')

In [ ]:
movies_df

In [ ]:
movies_df['year'] = movies_df['title'].str.extract('(\(\d{4}\))', expand=False)

In [ ]:
movies_df

In [ ]:
#REMOVER PARENTESIS DE LA COLUMNA 'YEAR'
movies_df['year'] = movies_df.year.str.extract('(\d{4})', expand = False)

In [ ]:
movies_df

In [ ]:
#REMOVER EL AÑO DE LA COLUMNA TITULO
movies_df['title'] = movies_df.title.str.replace(r"\s+\(.*\)", "", regex = True)

In [ ]:
movies_df

In [ ]:
#CONVERTIR LA COLUMNA DE GENEROS EN UNA LISTA(VECTOR)
movies_df['genres'] = movies_df['genres'].str.split('|')

In [ ]:
movies_df

In [ ]:
movies_df = movies_df[movies_df['genres'].apply(lambda g: '(no genres listed)' not in g)]

In [ ]:
#OBTENER UNA COPIA DEL DATAFRAME ORIGINAL
moviesWithGenres_df = movies_df.copy()

In [ ]:
#PARA CADA ROW EN EL DATAFRAME VAMOS A ITERAR SOBRE LOS GENEROS Y ASIGNAR UN 1 EN LA COLUMNA CORRESPONDIENTE

for index, row in movies_df.iterrows():
    for genres in row['genres']:
        if genres != '(no genres listed)':
            moviesWithGenres_df.at[index,genres] = 1
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df


In [ ]:
ratings_df

In [ ]:
#USANDO LAMBDA CON AXIS = 0 PARA ENCONTRAR MAXIMO DE RATINGS Y MINIMA DE RATINGS(POR COLUMNAS)
max_Ratings = ratings_df['rating'].max()
min_Ratings = ratings_df['rating'].min()
print(max_Ratings, min_Ratings)

In [ ]:
#BORRAR COLUMNA DE TIMESTAMP
ratings_df = ratings_df.drop('timestamp', axis = 1) 
ratings_df

In [ ]:
#ENTRADA DE DATOS

usuario_en = [
    {"title": "Breakfast Club, The", "rating": 4},
    {"title": "Toy Story", "rating": 5},
    {"title": "Jumanji", "rating": 3},
    {"title": "Pulp Fiction", "rating": 5},
    {"title": "Akira", "rating": 4.5}
]

In [ ]:
#CONVERTIR LA ENTRADA DE DATOS A DATAFRAME
entrada_peli = pd.DataFrame(usuario_en)

entrada_peli

In [ ]:
#VAMOS A AGREGAR EL ID A NUESTRAS PELICULAS DEL USUARIO, POR NOMBRE

id = movies_df[movies_df['title'].isin(entrada_peli['title'].tolist())]

entrada_peli = pd.merge(id, entrada_peli, how="inner", on = None, validate='many_to_many')

entrada_peli


In [ ]:
# ELIMINAMOS INFO INNECESARIA

entrada_peli = entrada_peli.drop('genres', axis = 1).drop('year', axis = 1)

entrada_peli

In [ ]:
#CODIFICAR LAS PELICULAS DEL USUARIO PARA ASIGNAR ONE HOT ENCODING

peli_usuario = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(entrada_peli['movieId'].tolist())]

peli_usuario

In [ ]:
peli_usuario = peli_usuario.reset_index(drop=True)
tabla_generos = peli_usuario.drop('movieId', axis = 1).drop('genres', axis=1).drop('title', axis=1).drop('year', axis=1)

tabla_generos

In [ ]:
#INICIAMOS EL MODELO DE LA MATRIX DE PESO
tabla_generos.transpose()
tabla_generos

In [ ]:
entrada_peli['rating']

In [ ]:
perfil_usu = tabla_generos.transpose().dot(entrada_peli['rating'])
perfil_usu

In [ ]:
#MOSTRAMOS LOS GENEROS DE LA TABLA ORIGINAL
generos = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])

generos

In [ ]:
generos = generos.drop('title', axis= 1).drop('year', axis= 1).drop('genres', axis= 1)

generos

In [ ]:
#REALIZAR PROMEDIO PONDERADO
recom = pd.DataFrame(generos['movieId'])
recom['ponderation'] = ((generos * perfil_usu).sum(axis=1)) / (perfil_usu.sum())

recom = pd.DataFrame(recom)

recom

In [ ]:
recom = recom.sort_values(by = 'ponderation', ascending= False)

recom.head(20)

In [ ]:
#TABLA FINAL DE RECOMENDACIONES
movies_df.set_index('movieId')
recom.set_index('movieId')
movies_df_position = movies_df.join(recom, lsuffix = '_key', on = 'movieId', how = 'right', validate='many_to_many')
movies_df_position = movies_df_position.sort_values(by='ponderation', ascending=False)
movies_df_position.head(20)
